# How to run Jaxions simulations with Python

In this notebook we briefly explain the syntax to prepare and run simulations from a python console/notebok.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#Pyaxions modules
from pyaxions import jaxions as pa
from pyaxions import randomstrings as rs
from pyaxions import simgen as sg

## Single network simulation

In [ ]:
N = 256
L = 6.
msa = 1.0 

#generate command line flags
rank, jax = sg.simgen(N=N, L=L, msa=msa, zRANKS=2, ic='lola', ict='lola', steps=20000, lap=2, dump = 20, dev = 'cpu', fftplan = 64, 
            nqcd=4.0, ctf=10.0, vqcd='vqcd0',  cti=0.01, meas= 0, sIter=10)

For all the options try help(sg.simgen) or look at the code in scripts/pyaxions/simgen.py!

In [ ]:
#run the simulation
sg.runsim(JAX=jax, MODE='run', RANK=rank, THR=1, USA='', VERB = True)

## Multiple Network Simulations

We wrote the function 'multisimgen' that can handle lists of input parameters. The only criterion is that if you put in multiple lists for parameters, they have to be the same length! It uses the i-th entry of all the lists for the i-th string of flags.

In [ ]:
Ns = [256, 512, 1024]
L = 6.
msas = [1.0, 0.5, 0.25]

#Creates a list of len(N) command line flags with different parameters
ranks, jaxs = sg.simgen(N=Ns, L=L, msa=msas, zRANKS=2, ic='lola', ict='lola', steps=20000, lap=2, dump = 20, dev = 'cpu', fftplan = 64, 
            nqcd=4.0, ctf=10.0, vqcd='vqcd0',  cti=0.01, meas= 0, sIter=10)

Now we can run again! The function 'multirun' can handle input lists the same way as 'multisimgen' and features a parameter 'STAT' in case you want to repeat the same simulation STAT times in order to get some statistics. In total this covers four cases:
1) Single configuration (in principle the same as the "old" runsim)
2) Single configuration with STAT repetitions
3) Multiple configurations with one simulation each
4) Multiple configurations with STAT repetitions each

In [ ]:
sg.multirun(JAX=jaxs, RANK=ranks ,THR=1,USA:='', STAT=2, NAME='test', VERB = True)

Note that 'multirun' handles the dynamical renaming of the 'out' folders and gives information about their names after finishing each simulation.

## Special Case: "string" initial conditions

The 'string' IC are bit more complex, as we have to create them in $N=256$ and then extrapolate to the desired grid size. That's why we developed 'runstring' which does the appropriate resclaling automatically. For information about the different types of ICs that can be generated at the moment consider looking at the 'scripts/pyaxions/randomstrings.py' module!

In [ ]:
N = 256
L = 6.
msa = 1.0 

RL = 0.25 # R/L: Radius of the loop in the simulation volume

In [ ]:
#Create string.dat file for circular loop (cf. randomstrings.py module)
x,y,z = rs.onestring(N=N, RL=RL, SHAPE='l')

plt.figure(figsize = (6,6))
plt.plot(x,y, marker = 'x')

plt.xlim(0,256)
plt.ylim(0,256)

In [ ]:
rank, jax = sg.simgen(N=N, L=L, msa=msa, zRANKS=2, ic='lola', ict='lola', steps=20000, lap=2, dump = 20, dev = 'cpu', fftplan = 64, 
            nqcd=4.0, ctf=10.0, vqcd='vqcd0',  cti=0.01, meas= 0, sIter=10)

Now we plug the command line flags in and let 'runstring' do the rescaling for us!

In [ ]:
sg.runstring(JAX=jax, RANK= rank, THR=1, USA=' ', OUT_CON='out_tmp', CON_OPTIONS ='--msa %f'%(msa))